In [50]:
c = chord.Chord('C2 F#2 B2 E3 A-3 C#4')
c.show('midi')

In [51]:
import copy
def octaveSwap(cIn):
    c = copy.deepcopy(cIn)
    for i in range(1, len(c.pitches) - 1):
        copyOctave = i + 1
        if copyOctave == len(c.pitches) - 1:
            copyOctave = 1
        c.pitches[i].octave = cIn.pitches[copyOctave].octave
    return c

c2 = octaveSwap(c)
c2.show('midi')

In [59]:
import random

c.volume = 60
s = stream.Stream([c])
s.append(dynamics.Dynamic('mp'))
cThis = c
for i in range(6):
    cThis = octaveSwap(cThis)
    s.append(cThis)

for j in range(8):
    cThis.volume.velocity *= 1.05
    cNext = copy.deepcopy(cThis)
    cNext.pitches[0].octave += 2
    cNext.sortAscending(inPlace=True)
    if j % 2 == 0:
        pNew = pitch.Pitch(random.randint(cNext.pitches[1].ps + 1, cNext.pitches[-1].ps - 1))
        cNext.pitches = cNext.pitches + (pNew,)
        cNext.sortAscending(inPlace=True)
    tupChoice = random.choice([(7, 5), (7, 6), (5, 4), (3, 2), (4, 5), (2,3)])
    cNext.duration.appendTuplet(duration.Tuplet(*tupChoice))
    s.append(cNext)
    cThis = cNext
    for i in range(random.randint(6, 14)):
        cThis = octaveSwap(cThis)
        if random.random() > .75:
            cThis.articulations.append(articulations.Accent())
        else:
            cThis.articulations = []
        s.append(cThis)
cOrig = copy.deepcopy(c)
cOrig.duration.type = 'whole'
cOrig.duration.dots = 1

s.append(copy.deepcopy(cOrig))
s.append(copy.deepcopy(cOrig))
s.append(copy.deepcopy(cOrig))
s.append(copy.deepcopy(cThis))
s.append(octaveSwap(cThis))
s.append(octaveSwap(octaveSwap(cThis)))
s.append(note.Rest('32nd'))
cOrig.articulations.append(articulations.StrongAccent())
s.append(cOrig)
s.show('midi')

In [49]:
sCh = stream.Part()
sCl = stream.Part()

for n in s.notes:
    nCopy = copy.deepcopy(n)
    nCopy.duration.tuplets = ()
    nCopy2 = copy.deepcopy(nCopy)
    if n.isRest:
        sCh.append(nCopy)
        sCl.append(nCopy)
    else:
        nCopy.pitches = [p for p in nCopy.pitches if p.ps > 61]
        nCopy2.pitches = [p for p in nCopy2.pitches if p.ps <= 61]
        if len(nCopy.pitches) > 0:
            sCh.append(nCopy)
        else:
            sCh.append(note.Rest())
        if len(nCopy2.pitches) > 0:
            sCl.append(nCopy2)
        else:
            sCl.append(note.Rest())
            
        
sCC = stream.Score()
sCC.insert(0, sCh)
sCC.insert(0, sCl)
sCC.write('musicxml')

'/var/folders/x5/rymq2tx16lqbpytwb1n_cc4c0000gn/T/music21/tmpqowkt1m6.xml'